## TEST with parlement speech embeddings

In [1]:
from annoy import AnnoyIndex
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow_text
import tensorflow_hub as hub

In [2]:
embeddings = pickle.load(open("kamer_debatten_recente_speeches_met_emb.pck", "rb"))

In [3]:
embeddings

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,...,502,503,504,505,506,507,508,509,510,511
0,2010-01-12,NaN,1,weekers,VVD,828.0,False,376,Voorzitter. In de week van de val van DSB heef...,NL-TweedeKamer,...,0.053838,-0.041742,-0.032260,-0.067037,-0.022552,-0.023394,-0.065473,-0.069716,0.012569,0.062962
1,2010-01-12,NaN,2,bos,PvdA,1234.0,False,260,Voorzitter. De laatste conclusie van de heer W...,NL-TweedeKamer,...,0.053762,-0.070165,-0.014723,-0.067824,-0.056783,-0.006215,-0.072602,-0.071131,-0.018375,0.080761
2,2010-01-12,NaN,3,weekers,VVD,828.0,False,204,Het gaat ons er niet om dat de minister zich b...,NL-TweedeKamer,...,0.069975,-0.066717,-0.014065,-0.068072,-0.066792,0.006887,-0.068825,-0.071827,0.018592,0.074508
3,2010-01-12,NaN,4,bos,PvdA,1234.0,False,226,Ik zal die zorgen overbrengen aan de heer Sche...,NL-TweedeKamer,...,0.082787,-0.077583,0.064040,-0.065484,-0.048932,-0.053128,-0.070319,-0.073950,0.039685,0.095769
4,2010-01-12,NaN,5,tony van dijck,PVV,298.0,False,107,Ook de fractie van de Partij voor de Vrijheid ...,NL-TweedeKamer,...,-0.019395,-0.025600,-0.051116,-0.067381,-0.071851,-0.021703,-0.082825,-0.032184,-0.049826,0.088014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468192,2019-07-04,NaN,963,staatssecretaris broekers-knol,NaN,NaN,False,290,Ik ga uit van de gegevens die wij hebben gekre...,NL-TweedeKamer,...,0.051056,-0.042262,-0.051334,-0.051260,-0.016387,-0.056653,-0.002518,-0.075303,-0.005929,0.087212
468193,2019-07-04,NaN,965,staatssecretaris broekers-knol,NaN,NaN,False,142,Dan de motie op stuk nr. 2519 van de heer Hidd...,NL-TweedeKamer,...,0.059314,-0.056630,-0.031098,-0.041214,-0.001191,0.002085,0.038942,-0.011882,0.044419,0.059911
468194,2019-07-04,NaN,967,de voorzitter,NaN,NaN,True,62,Over exact 60 minuten gaan wij stemmen over de...,NL-TweedeKamer,...,0.008260,-0.023593,0.078514,-0.011184,-0.030405,-0.023197,0.012507,-0.052127,0.023723,0.062293
468195,2019-07-04,NaN,968,mevrouw van toorenburg,CDA,1157.0,False,21,Toch nog even — misschien is het allemaal afge...,NL-TweedeKamer,...,-0.073596,0.022221,-0.085983,-0.041239,0.000995,-0.014434,-0.021121,-0.051808,0.041230,0.124054


## afstanden tussen rijen met scikit learn

In [10]:
def find_speeches(i:int, historic_speeches, K:int = 10) :
    
    testquery = historic_speeches.iloc[i:(i+1),12:524]
   
    #### look for the closest article embeddings, that we just created above
    #### the results object is an array with nart cosine similarities
    results = cosine_similarity(historic_speeches.iloc[:,12:524], testquery)

    indices = results.argsort(axis = 0).ravel()
    best_K = indices[-K:]

    ### take corresponding articles
    out = historic_speeches.iloc[best_K,:]
    out = (
        out
        .assign(similarity = results[best_K].flatten())
        .sort_values(by = "similarity", ascending = False)
    )
    return out[["datum", "speaker", "party", "text", "similarity"]]
    

In [11]:
%%time 
find_speeches(11, embeddings)

CPU times: user 1.74 s, sys: 766 ms, total: 2.5 s
Wall time: 2.13 s


,datum,speaker,party,text,similarity
11,2010-01-12,bos,PvdA,Mevrouw de voorzitter. Mevrouw Verdonk is een ...,1.000000
1,2010-01-12,bos,PvdA,Voorzitter. De laatste conclusie van de heer W...,0.764701
445203,2019-02-12,minister grapperhaus,NaN,Het is misschien goed om daar toch nog het vol...,0.702756
105887,2012-03-07,schippers,VVD,"Nee, ik heb de Kamer een onderzoek voorgesteld...",0.701216
117142,2012-05-15,aptroot,VVD,"Ja, ik rond af. Ik heb een laatste punt over d...",0.698580
300100,2015-12-16,minister van der steur,NaN,Ik stel vast dat de commissie-Oosting dat ook ...,0.695527
456505,2019-04-17,staatssecretaris keijzer,NaN,Ik zal mijn uiterste best voor u doen. In de m...,0.684091
99128,2012-01-31,opstelten,CDA,Op verzoek van de Kamer en in goed overleg met...,0.680848
302296,2016-01-20,minister van der steur,NaN,Ik vind dat de Kamer geïnformeerd moet worden....,0.679452
184499,2013-10-30,schippers,VVD,"Over de discussie, de debatten en de agenderin...",0.678469


## Now lets try annoy

In [33]:
f = 512
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed

In [34]:
embeddings.iloc[1:2,12:524].values.flatten().shape

(512,)

In [35]:
N = embeddings.shape[0]
N

468197

In [36]:
%%time 
for i in range(N):
    v = embeddings.iloc[i:(i+1),12:524].values.flatten()
    t.add_item(i, v)

CPU times: user 4min 44s, sys: 3.48 s, total: 4min 48s
Wall time: 4min 48s


In [39]:
%%time
t.build(50)
t.save('test.ann')


True

In [7]:
%%time 
f = 512
u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, will just mmap the file


CPU times: user 233 µs, sys: 307 µs, total: 540 µs
Wall time: 6.2 ms


True

In [14]:
%%time 

best_K = u.get_nns_by_item(11, 10)
out = embeddings.iloc[best_K,:]
out[["datum", "speaker", "party", "text"]]
    

CPU times: user 3.02 ms, sys: 1.2 ms, total: 4.22 ms
Wall time: 2.78 ms


,datum,speaker,party,text
11,2010-01-12,bos,PvdA,Mevrouw de voorzitter. Mevrouw Verdonk is een ...
105887,2012-03-07,schippers,VVD,"Nee, ik heb de Kamer een onderzoek voorgesteld..."
300100,2015-12-16,minister van der steur,NaN,Ik stel vast dat de commissie-Oosting dat ook ...
99128,2012-01-31,opstelten,CDA,Op verzoek van de Kamer en in goed overleg met...
448200,2019-03-05,minister hoekstra,NaN,Daar staat dan tegenover dat we bij die overwe...
6276,2010-02-16,balkenende,CDA,De commissie heeft uit eigen beweging gezegd: ...
36580,2010-12-09,kamp,VVD,Geen haar op mijn hoofd die eraan denkt om bij...
266362,2015-04-29,van der steur,VVD,"Dat staat er inderdaad, maar de vraag rijst da..."
264382,2015-04-21,van rijn,PvdA,Toch even precies en naar mijn smaak zorgvuldi...
460040,2019-05-21,minister hoekstra,NaN,"Ik aarzel daar, puur omdat ik dan even zou moe..."


In [15]:
%%time 
find_speeches(11, embeddings)

CPU times: user 1.66 s, sys: 731 ms, total: 2.4 s
Wall time: 1.98 s


,datum,speaker,party,text,similarity
11,2010-01-12,bos,PvdA,Mevrouw de voorzitter. Mevrouw Verdonk is een ...,1.000000
1,2010-01-12,bos,PvdA,Voorzitter. De laatste conclusie van de heer W...,0.764701
445203,2019-02-12,minister grapperhaus,NaN,Het is misschien goed om daar toch nog het vol...,0.702756
105887,2012-03-07,schippers,VVD,"Nee, ik heb de Kamer een onderzoek voorgesteld...",0.701216
117142,2012-05-15,aptroot,VVD,"Ja, ik rond af. Ik heb een laatste punt over d...",0.698580
300100,2015-12-16,minister van der steur,NaN,Ik stel vast dat de commissie-Oosting dat ook ...,0.695527
456505,2019-04-17,staatssecretaris keijzer,NaN,Ik zal mijn uiterste best voor u doen. In de m...,0.684091
99128,2012-01-31,opstelten,CDA,Op verzoek van de Kamer en in goed overleg met...,0.680848
302296,2016-01-20,minister van der steur,NaN,Ik vind dat de Kamer geïnformeerd moet worden....,0.679452
184499,2013-10-30,schippers,VVD,"Over de discussie, de debatten en de agenderin...",0.678469
